Real quick recap, assembly language is a 1-1 translation of binary and high level code. The instructions you send to the CPU all have an OPCODE or a set of bits specifiying what the instruciton is, such as ADD. These instructions are followed by arguments or the things you supply after the instruction and the instructions fall into 4 general categories; arithmetic, logical, data transfer, and (un)conditional instructions. It seems like the instructions which are immediate (like addi) take one of their arguments from a register inside the CPU; makes sense since it is almost immediate or in the immediate vicinity. There is also no subi since addi can add a register and any number be it positive or negative. Remember, the default bias values is **127**. 

**Bitwise operations** are fast and simple instructions that operate on binary number's individual bits. Remember that since most of the bits in our numbers won't be used, sign extension (replacing unused bits with sign bit) will be used. So, yes, immediate numbers or arguments are not loaded from main memory but are used immediately in the program (a constant value or constant expression; one that you define in the instruction). **Directives** are 